In [1]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [2]:
gnn_train_data = pd.read_parquet('../data/curated/ML_data/gnn_train_data.parquet')
gnn_val_data = pd.read_parquet('../data/curated/ML_data/gnn_val_data.parquet')
gnn_test_data = pd.read_parquet('../data/curated/ML_data/gnn_test_data.parquet')

SA2_gnn_data = pd.read_parquet('../data/curated/ML_data/SA2_gnn_data.parquet')
station_inference_gnn_data = pd.read_parquet('../data/curated/ML_data/station_inference_gnn_data.parquet')

station_inference_gnn_data = station_inference_gnn_data.rename({'Station_Na': 'Station Name'}, axis=1)
inference_data = pd.concat([SA2_gnn_data, station_inference_gnn_data], axis=0)
inference_data = inference_data.rename({'Station Name': 'Station_Name'}, axis=1)

In [3]:
# stations_list = [x for x in gnn_train_data['Station_Name'].unique()]
# stations_index = {stations_list[i]:i for i in range(len(stations_list))}
# reverse_stations_index = {v: k for (k, v) in stations_index.items()}

# open npy
station_weights_matrix = np.load('../data/curated/ML_data/station_weights_matrix.npy')
SA2_weights_matrix = np.load('../data/curated/ML_data/station_weights_withSA2_matrix.npy')

with open('../data/curated/ML_features/station_weights_withSA2.json', 'r') as f:
    station_weights_withSA2 = json.load(f)

with open('../data/curated/ML_features/station_weights.json', 'r') as f:
    station_weights = json.load(f)

In [4]:
geospatial_features = ['log_Total_Demand']
non_geospatial_features = ['Weekday', 'PublicHoliday', 'mean_rainfall_value', 'has_school',
       'has_sport_facility', 'has_shopping_centre', 'has_hospital',
       'total_population', ' med_rent_weekly_c2021',
       ' med_mortg_rep_mon_c2021', ' med_person_inc_we_c2021',
       ' med_famly_inc_we_c2021']
label_columns = ['log_Total_Demand']

In [5]:
def DataFactory(raw_dataset, geospatial_features, non_geospatial_features, label_columns, stations_index, inference = False):

    """ Data Factory of GNN """
    
    geospatial_x_batches = []
    non_geospatial_x_batches = []
    y_batches = []
    masks = []

    if inference:
        groupby_column = 'Weekday'
    else:
        groupby_column = 'Business_Date'

    for day, daily_df in tqdm(raw_dataset.groupby([groupby_column])):

        geospatial_x = np.zeros([len(stations_index), len(geospatial_features)])
        y = np.zeros([len(stations_index), len(label_columns)])
        mask = np.zeros([len(stations_index), 1])
        non_geospatial_x = np.zeros([len(stations_index), len(non_geospatial_features)])

        daily_df.set_index('Station_Name', inplace=True)

        for station in daily_df.index:

            geospatial_x[stations_index[station]] = daily_df.loc[station][geospatial_features] # todo inference. 
            if not inference:
                y[stations_index[station]] = daily_df.loc[station][label_columns]
            mask[stations_index[station]] = 1
            non_geospatial_x[stations_index[station]] = daily_df.loc[station][non_geospatial_features]
                
        geospatial_x_batches.append(geospatial_x)
        y_batches.append(y)
        masks.append(mask.flatten())

        non_geospatial_x_batches.append(non_geospatial_x)

        
    return geospatial_x_batches, non_geospatial_x_batches, y_batches, masks

In [6]:
train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks = DataFactory(gnn_train_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks = DataFactory(gnn_val_data, geospatial_features, non_geospatial_features, label_columns, station_weights)
test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks = DataFactory(gnn_test_data, geospatial_features, non_geospatial_features, label_columns, station_weights)

# inference_geospatial_X_batches, inference_non_geospatial_X_batches, inference_y_batches, inference_masks = DataFactory(inference_data, geospatial_features, non_geospatial_features, label_columns, station_weights_withSA2, inference = True)

100%|██████████| 82/82 [00:18<00:00,  4.50it/s]


In [7]:
import sys
import os

py_file_location = '../'
home_directory = '../'

sys.path.append(os.path.abspath(py_file_location))
from model.model_class.environment import *

In [8]:
from model.model_class import GNN

In [9]:
SEED = 42

In [19]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0.1
    n_gnn_layers = 1
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

100%|██████████| 382/382 [00:14<00:00, 25.52it/s]


 Epoch 1 Train | Loss:  0.0865 | R2:  0.9061| MSE:  0.0862 | RMSE:  0.2935 | MAE:  0.1932 


100%|██████████| 82/82 [00:00<00:00, 117.21it/s]


Epoch 1 Val | Loss:  0.0131 | R2:  0.9868| MSE:  0.0131 | RMSE:  0.1146 | MAE:  0.0860 


100%|██████████| 382/382 [00:13<00:00, 27.51it/s]


...

Epoch 9 Val | Loss:  0.0037 | R2:  0.9963| MSE:  0.0037 | RMSE:  0.0611 | MAE:  0.0476 





100%|██████████| 82/82 [00:00<00:00, 95.64it/s] 


Epoch 6 Val | Loss:  0.0010 | R2:  0.9990| MSE:  0.0010 | RMSE:  0.0310 | MAE:  0.0245 


100%|██████████| 82/82 [00:00<00:00, 104.91it/s]


Epoch 6 Val | Loss:  0.0009 | R2:  0.9990| MSE:  0.0009 | RMSE:  0.0303 | MAE:  0.0245 


In [14]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0
    n_gnn_layers = 1
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

  0%|          | 0/382 [00:00<?, ?it/s]

100%|██████████| 382/382 [00:15<00:00, 25.46it/s]


 Epoch 1 Train | Loss:  0.0811 | R2:  0.9121| MSE:  0.0806 | RMSE:  0.2840 | MAE:  0.1871 


100%|██████████| 82/82 [00:01<00:00, 75.52it/s]


Epoch 1 Val | Loss:  0.0118 | R2:  0.9882| MSE:  0.0118 | RMSE:  0.1086 | MAE:  0.0862 


...

Epoch 12 Val | Loss:  0.0006 | R2:  0.9994| MSE:  0.0006 | RMSE:  0.0248 | MAE:  0.0151 





100%|██████████| 82/82 [00:00<00:00, 93.37it/s]


Epoch 10 Val | Loss:  0.0004 | R2:  0.9996| MSE:  0.0004 | RMSE:  0.0194 | MAE:  0.0127 


100%|██████████| 82/82 [00:00<00:00, 89.12it/s]


Epoch 10 Val | Loss:  0.0003 | R2:  0.9996| MSE:  0.0003 | RMSE:  0.0182 | MAE:  0.0120 


In [15]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0
    n_gnn_layers = 2
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

100%|██████████| 382/382 [00:26<00:00, 14.38it/s]


 Epoch 1 Train | Loss:  0.1012 | R2:  0.8902| MSE:  0.1007 | RMSE:  0.3174 | MAE:  0.2074 


100%|██████████| 82/82 [00:01<00:00, 55.75it/s]


Epoch 1 Val | Loss:  0.0198 | R2:  0.9801| MSE:  0.0198 | RMSE:  0.1407 | MAE:  0.1138 


100%|██████████| 382/382 [00:22<00:00, 17.07it/s]


...

Epoch 6 Val | Loss:  0.0481 | R2:  0.9518| MSE:  0.0481 | RMSE:  0.2193 | MAE:  0.1825 





100%|██████████| 82/82 [00:01<00:00, 60.71it/s]


Epoch 1 Val | Loss:  0.0198 | R2:  0.9801| MSE:  0.0198 | RMSE:  0.1407 | MAE:  0.1138 


100%|██████████| 82/82 [00:01<00:00, 70.01it/s]


Epoch 1 Val | Loss:  0.0212 | R2:  0.9764| MSE:  0.0212 | RMSE:  0.1455 | MAE:  0.1176 


In [18]:
class GNN_config:
    # ----------------- architectual hyperparameters ----------------- #
    d_model = 256
    n_heads = 8
    dropout = 0
    n_gnn_layers = 3
    activation = nn.ReLU()
    res_learning = False
    bottleneck = True
    # ----------------- optimisation hyperparameters ----------------- #
    random_state = SEED
    epochs = 32
    lr = 1e-3
    patience = 5
    loss = nn.MSELoss()
    validation_loss = nn.MSELoss()
    alpha = 0.1
    scheduler = True
    grad_clip = False
    # ----------------- operation hyperparameters ----------------- #
    spatial_input_dim = 1
    nonspatial_input_dim = 12
    # ----------------- saving hyperparameters ----------------- #
    rootpath = home_directory
    name = f'GNN'

model = GNN(GNN_config) # initialise the model

# train the model (all cells except this one will print training log and evaluation at each batch)
best_epoch = model.fit(train_geospatial_X_batches, train_non_geospatial_X_batches, train_y_batches, train_masks, val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix)
print('\n\n')

# as model automatically saves best epoch, will now load the best epoch and evaluate on test set
model.load()
model.eval(val_geospatial_X_batches, val_non_geospatial_X_batches, val_y_batches, val_masks, station_weights_matrix, best_epoch, evaluation_mode = True)
model.eval(test_geospatial_X_batches, test_non_geospatial_X_batches, test_y_batches, test_masks, station_weights_matrix, best_epoch, evaluation_mode = True)

100%|██████████| 382/382 [00:37<00:00, 10.24it/s]


 Epoch 1 Train | Loss:  0.1168 | R2:  0.8731| MSE:  0.1164 | RMSE:  0.3411 | MAE:  0.2250 


100%|██████████| 82/82 [00:03<00:00, 27.12it/s]


Epoch 1 Val | Loss:  0.0469 | R2:  0.9530| MSE:  0.0469 | RMSE:  0.2165 | MAE:  0.1852 


100%|██████████| 382/382 [00:34<00:00, 11.10it/s]


...

Epoch 9 Val | Loss:  0.0110 | R2:  0.9889| MSE:  0.0110 | RMSE:  0.1050 | MAE:  0.0882 





100%|██████████| 82/82 [00:02<00:00, 36.37it/s]


Epoch 6 Val | Loss:  0.0085 | R2:  0.9915| MSE:  0.0085 | RMSE:  0.0921 | MAE:  0.0657 


100%|██████████| 82/82 [00:02<00:00, 38.78it/s]


Epoch 6 Val | Loss:  0.0092 | R2:  0.9897| MSE:  0.0092 | RMSE:  0.0960 | MAE:  0.0668 
